In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import logging
import arviz as az

In [2]:
# Set up logging: messages will include timestamp, log level, and message content
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

PATH_MODEL = '/volumes/hyijie_psy/CPP_stage2_HDDM/mid_van_2019/model_fitted'
PATH_MODEL_FIt = '../results/ddm'
os.makedirs(PATH_MODEL_FIt, exist_ok=True)

MODEL_CONFIG = {
    #'m2_ams': {
    #    'main_param': 'v_ams',
    #    'interactions': [
    #        'v_C(condition)[T.2.0]:ams',
    #        'v_C(condition)[T.3.0]:ams',
    #        'v_C(condition)[T.4.0]:ams'
    #    ]
    #},
    #'m2_pams': {
    #    'main_param': 'v_pams',
    #    'interactions': [
    #        'v_C(condition)[T.2.0]:pams',
    #        'v_C(condition)[T.3.0]:pams',
    #        'v_C(condition)[T.4.0]:pams'
    #    ]
    #},
    #'m2_slps': {
    #    'main_param': 'v_slps',
    #    'interactions': [
    #        'v_C(condition)[T.2.0]:slps',
    #        'v_C(condition)[T.3.0]:slps',
    #        'v_C(condition)[T.4.0]:slps'
    #    ]
    #},
    'm2_ams_bin': {
        'main_param': 'v_ams_bin',
        'interactions': [
            'v_C(condition)[T.2.0]:ams_bin',
            'v_C(condition)[T.3.0]:ams_bin',
            'v_C(condition)[T.4.0]:ams_bin'
        ]
    },
    'm2_pams_bin': {
        'main_param': 'v_pam_bin',
        'interactions': [
            'v_C(condition)[T.2.0]:pam_bin',
            'v_C(condition)[T.3.0]:pam_bin',
            'v_C(condition)[T.4.0]:pam_bin'
        ]
    }#,
    #'m2_slps_bin': {
    #    'main_param': 'v_slp_bin',
    #    'interactions': [
    #        'v_C(condition)[T.2.0]:slp_bin',
    #        'v_C(condition)[T.3.0]:slp_bin',
    #        'v_C(condition)[T.4.0]:slp_bin'
    #    ]
    #}
}

In [3]:
# Extract and save posterior and Loo value

loo_results = []

for model_name, config in MODEL_CONFIG.items():

    main_param = config['main_param']
    interaction_names = config['interactions']
    
    path_model_infdata = os.path.join(PATH_MODEL, model_name + '.nc')
    assert os.path.exists(path_model_infdata), f" nc file not found: {model_name}"
    
    logger.info(f"Processing model: {model_name}")

    # Extract and save Loo for all models
    m_infdata = az.from_netcdf(path_model_infdata)
    loo = az.loo(m_infdata, pointwise=True)
    loo_dict = {
            "model": main_param,
            "loo": loo.elpd_loo,
            "p_loo": loo.p_loo
        }
    loo_results.append(loo_dict)

    # Save posterior data for all models
    posterior_dict = {}

    # Main effect
    if main_param in m_infdata.posterior:
        posterior_dict[main_param] = m_infdata.posterior[main_param].stack(sample=("chain", "draw")).values
    else:
        logger.error(f"Main parameter '{main_param}' not in {model_name}")
        continue
    # Interaction effect
    for inter in interaction_names:
        if inter in m_infdata.posterior:
            posterior_dict[inter] = m_infdata.posterior[inter].stack(sample=("chain", "draw")).values
        else:
            logger.warning(f"Interaction '{inter}' not found in model '{model_name}'")

    # Save data
    data_posterior = pd.DataFrame(posterior_dict)
    path_save_posterior = os.path.join(PATH_MODEL_FIt, f'{model_name}_posterior.csv')
    data_posterior.to_csv(path_save_posterior, index=False)

loo_df = pd.DataFrame(loo_results)
loo_df.to_csv(os.path.join(PATH_MODEL_FIt, "loo_comparison.csv"), index=False)

2025-12-26 14:11:26,303 - INFO - Processing model: m2_ams_bin
/opt/miniconda3/envs/env4eeg/lib/python3.13/site-packages/arviz/stats/stats.py:1042: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/opt/miniconda3/envs/env4eeg/lib/python3.13/site-packages/arviz/stats/stats.py:1074: RuntimeWarning: overflow encountered in expm1
  x = np.expm1(-kappa * np.log1p(-probs)) / kappa
/opt/miniconda3/envs/env4eeg/lib/python3.13/site-packages/numpy/_core/_methods.py:51: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/opt/miniconda3/envs/env4eeg/lib/python3.13/site-packages/arviz/stats/stats.py:782: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very differen

In [5]:

path_model_infdata = os.path.join(PATH_MODEL, 'm2_slps.nc')
assert os.path.exists(path_model_infdata), f" nc file not found: {model_name}"

logger.info(f"Processing model: {model_name}")
# Extract and save Loo for all models
m_infdata = az.from_netcdf(path_model_infdata)

2025-12-26 12:54:44,082 - INFO - Processing model: m2_slps_bin


In [6]:
az.summary(m_infdata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,1.277,0.040,1.203,1.354,0.000,0.000,11419.0,12279.0,1.00
a_std,0.177,0.033,0.120,0.239,0.000,0.000,8265.0,10539.0,1.00
a_subj.14,1.190,0.060,1.083,1.302,0.001,0.000,3823.0,9663.0,1.00
a_subj.15,1.316,0.050,1.224,1.411,0.000,0.000,11432.0,13428.0,1.00
a_subj.16,1.473,0.076,1.338,1.623,0.001,0.001,2950.0,3679.0,1.00
...,...,...,...,...,...,...,...,...,...
v_C(condition)[T.4.0]:slps_subj.32,0.259,0.216,-0.102,0.687,0.011,0.002,331.0,1542.0,1.02
v_C(condition)[T.4.0]:slps_subj.33,0.142,0.190,-0.208,0.521,0.010,0.003,392.0,2840.0,1.01
v_C(condition)[T.4.0]:slps_subj.34,-0.008,0.187,-0.369,0.345,0.008,0.002,534.0,3747.0,1.01
v_C(condition)[T.4.0]:slps_subj.35,-0.040,0.209,-0.458,0.335,0.008,0.003,711.0,4098.0,1.01


In [7]:
m_infdata

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> prior
	> observed_data